In [ ]:
# !pip install pororo -q

In [ ]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import re

from datetime import datetime, timedelta
import time 
from tqdm import trange

#### 📄제품 LIst 확보

In [ ]:
# URL 예시
main = 'https://makers.kakao.com/'
product_list = 'https://makers-gateway.kakao.com/service/front/v1/search?page=0&query=매일유업&size=50&on_sale=false' #json
review_list  = 'https://store.kakao.com/a/maeilecommerce/reviews?page=0&productId=97590058&size=20' #json

In [ ]:
code = []
search_keyword = [
                  # '매일유업',
                  '셀렉스', ]
for text_ in search_keyword:
    url_master = 'https://makers-gateway.kakao.com/service/front/v1/search?page=0&query='+text_+'&size=50&on_sale=false'
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.141 Safari/537.36'}
    ka_makers = requests.get(url_master, headers=headers).json()

    for i in range( len(ka_makers['result']['searchItemList']) ):
        category =  ka_makers['result']['searchItemList'][i]['categoryLargeName']
        product = ka_makers['result']['searchItemList'][i]['roundItemName']
        if ((category =='푸드') & (text_ in product) ) :
            code_ = ka_makers['result']['searchItemList'][i]['masterItemId']
            code.append(code_)
        else:
            pass

In [ ]:
code[:2]

[100011606, 100008290]

In [ ]:
url_master = 'https://makers-gateway.kakao.com/service/front/v1/search?page=0&query=셀렉스&size=50&on_sale=false'
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.141 Safari/537.36'}
ka_makers = requests.get(url_master, headers=headers).json()

In [ ]:
ka_makers['result']['searchItemList'][0]

{'masterItemId': 100011606,
 'roundItemId': 100056348,
 'roundItemName': '셀렉스 프로틴 음료 190ml x 24팩 (오리지널/로우슈거)',
 'brandId': 100000869,
 'brandName': '셀렉스',
 'brandNickname': 'selecs',
 'salesStatus': 'ISS4',
 'limitDisplay': False,
 'categorySetName': '주 카테고리',
 'categoryLargeId': 102,
 'categoryLargeName': '푸드',
 'categoryMediumId': 192,
 'categoryMediumName': '음료／커피／차',
 'categorySmallId': 241,
 'categorySmallName': '음료',
 'releasedAt': '2023-05-12 17:00:00',
 'imageUrl': '/makers_smith/file/items/100011606/masters/75cb801b25a247dbb1365e4c14872075.jpg',
 'mainImageUrl': '/makers_smith/file/items/100011606/masters/7cb04e38141e48b78623a3a567d23633.jpg',
 'thumbnailImageUrl': '/makers_smith/file/items/100011606/masters/043a3c6a6abe42f391235178e1de5bad.jpg',
 'accumulatedSoldCount': 17197,
 'itemPrice': 26900,
 'itemContents': '{"anchorSections":[],"detailDescription":[{"compId":86,"compType":"image","compProp":{"fileName":"1c3a3be3a16f4a11b02b610f0b9c3f93.jpg","filePath":"/makers_smith/f

#### 🅰제품 리뷰 수집

In [ ]:
url = 'https://makers-gateway.kakao.com/service/front/v2/reviews/item/' + str(code[1]) + '?reviewSort=LATEST&page=1&size=50'
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.141 Safari/537.36'}
ka_makers = requests.get(url, headers=headers).json()

i = 19
ka_makers['result']['contents'][i]

{'reviewId': 4774872,
 'masterItemId': 100008290,
 'roundItemId': 100038718,
 'cartId': 112303897,
 'roundItemName': '셀렉스 데이 & 나이트 다이어트 700mg x 180정',
 'roundItemImageUrl': '/makers_smith/file/items/100008290/masters/8fc0fd3d02b14e4a9f68811ae2f6e526.jpg',
 'userId': 1737639,
 'userProfileImageUrl': 'http://k.kakaocdn.net/dn/MqjAc/btseo15YE0p/s4bhwc1OoRqkkUhU6787nK/img_640x640.jpg',
 'userProfileName': '.',
 'sendUserId': None,
 'sendUserProfileName': None,
 'receiveUserId': None,
 'receiveUserProfileName': None,
 'score': 4,
 'likeCount': 0,
 'calculatedScore': 10.49,
 'text': '아침에  화장실  잘 가요  조금씩 몸무게가  줄어 들고 있어요',
 'status': 'NORMAL',
 'type': 'SELF',
 'owner': False,
 'comment': None,
 'createdAt': '2022-08-18 08:06:18',
 'updatedAt': '2022-08-18 08:06:18',
 'imageUrls': [],
 'latestRoundItemId': None,
 'latestRoundItemName': None,
 'latestRoundItemImageUrl': None,
 'limitUnder19': False}

In [ ]:
date_start = (datetime.today() + timedelta(hours=9) - timedelta(3)).replace(minute=0, hour=0, second=0, microsecond=0).strftime('%Y-%m-%d')
date_end   = (datetime.today() + timedelta(hours=9) - timedelta(0)).replace(minute=0, hour=0, second=0, microsecond=0).strftime('%Y-%m-%d')

In [ ]:
df = pd.json_normalize(ka_makers['result']['contents'])
df = df[['userId', 'createdAt', 'roundItemName', 'text', 'score', 'roundItemId']]
df = df.rename(columns ={'userId': "USER", 'roundItemName': "PRODUCT", 'text': "REVIEW", 'score': "SCORE", 'createdAt' : 'DATE', 'roundItemId' : 'GUBUN' })

df.REVIEW = df.REVIEW.str.replace('\n', ' ')
df['DATE'] = pd.to_datetime(df['DATE'], format = '%Y-%m-%d' )
df.GUBUN = df.GUBUN.astype('object')
df.USER  = df.USER.astype('object')


In [ ]:

# df = df[ (date_start <= df['DATE'] ) &  (date_end > df['DATE'] )].reset_index(drop=True)

In [ ]:
df

,USER,DATE,PRODUCT,REVIEW,SCORE,GUBUN
0,1318037,2023-03-13 21:53:14,셀렉스 데이 & 나이트 다이어트 700mg x 180정,좋습니다..,3,100040126
1,493061,2023-03-01 21:32:38,셀렉스 데이 & 나이트 다이어트 700mg x 180정,효과가 바로 나진 않아요 꾸준히 먹음 안먹을때보다는 좋긴해요 쉽게~,3,100048842
2,18425528,2023-02-14 01:14:59,셀렉스 데이 & 나이트 다이어트 700mg x 180정,저는 이 제품이 변비약 이라고 생각하오니 잘 판단하시고 구매하시기바랍니다. 처음엔 ...,1,100048842
3,2826247,2023-02-11 09:08:15,셀렉스 데이 & 나이트 다이어트 700mg x 180정,아직 하루지나서 잘 모르겠어요 우선 화장실은 잘가서 좋으네요,4,100049802
4,9060492,2023-02-08 01:11:04,셀렉스 데이 & 나이트 다이어트 700mg x 180정,"포장상태가 조금 안좋네요... 카카오메이커스 처음 이용해보았는데, 원래 다이런가요??",3,100049802
5,1336072,2023-01-31 20:30:26,셀렉스 데이 & 나이트 다이어트 700mg x 180정,아직 먹어보기 전이라 제품레 대해서는 말할 수 없지만 개수가 모자랍니다!! 나이트는...,2,100048842
6,1221567,2023-01-15 08:28:51,셀렉스 데이 & 나이트 다이어트 700mg x 180정,제가 장이 둔한편이라 밀가루음식만 먹으면 더부룩하고 화장실 잘 못가는 편이에요 근데...,4,100045340
7,229766,2022-12-27 13:39:22,셀렉스 데이 & 나이트 다이어트 700mg x 180정,별점 주기도 아까워서리...저만 효과를 못보는건지?효과 전혀 없네요,1,100045340
8,13524562,2022-12-12 11:15:45,셀렉스 데이 & 나이트 다이어트 700mg x 180정,가성비좋은 다이어트제품을 접하게되어 좋아요 앞으로도 더 좋은 제품 기다릴께요~,4,100045340
9,19867129,2022-12-12 08:43:41,셀렉스 데이 & 나이트 다이어트 700mg x 180정,도움 많이 되었어요 거의 다 먹어가서 재구입 예정입니당,4,100039412


In [ ]:
i = 17
user_id = ka_makers['result']['contents'][i]['userId']
reg_dtm = ka_makers['result']['contents'][i]['createdAt']
channel = 'kakaomakers'
product = ka_makers['result']['contents'][i]['roundItemName']
review  = ka_makers['result']['contents'][i]['text'].replace('\n', ' ')
score   = int(ka_makers['result']['contents'][i]['score']) 
gubun   = ka_makers['result']['contents'][i]['roundItemId']
if score >= 3:
  score = score + 1

user_id, reg_dtm, channel, product, review, score

(4129377,
 '2022-06-25 18:11:49',
 'kakaomakers',
 '매일유업 어메이징 오트 바리스타 950ml 3팩',
 '우선 진한 맛에 부드러웠고 커피를 타니 라떼맛이 났어요. 한참전에 먹었는데 이번 190ml주문하며 후기 쓰네요. 여름이 되니 그 라떼맛이 생각 나요',
 5)

In [ ]:
date_start = (datetime.today() + timedelta(hours=9) - timedelta(1)).replace(minute=0, hour=0, second=0).strftime('%Y-%m-%d %H:%M:%S')
date_end   = (datetime.today() + timedelta(hours=9) - timedelta(0)).replace(minute=0, hour=0, second=0).strftime('%Y-%m-%d %H:%M:%S')
date_start, date_end

('2022-07-12 00:00:00', '2022-07-13 00:00:00')

In [ ]:
https://makers-gateway.kakao.com/service/front/v2/reviews/item/100037426?reviewSort=LATEST&page=1&size=50

In [ ]:
def kakaomakers_review(code_):
    url = 'https://makers-gateway.kakao.com/service/front/v2/reviews/item/' + str(code_) + '?reviewSort=LATEST&page=1&size=50'
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.141 Safari/537.36'}
    ka_makers = requests.get(url, headers=headers).json()
    ka_makers = ka_makers['result']['contents']
    date_start = (datetime.today() + timedelta(hours=9) - timedelta(1)).replace(minute=0, hour=0, second=0).strftime('%Y-%m-%d %H:%M:%S')
    date_end   = (datetime.today() + timedelta(hours=9) - timedelta(0)).replace(minute=0, hour=0, second=0).strftime('%Y-%m-%d %H:%M:%S')
    for data_ in ka_makers:
        reg_dtm = data_['createdAt']
        if date_start <= reg_dtm  <= date_end :
            user_id = data_['userId']
            channel = 'kakaomakers'
            product = data_['roundItemName']
            review  = data_['text'].replace('\n', ' ')
            score   = int(data_['score']) 
            if score >= 3:
              score = score + 1
            print([user_id, reg_dtm, channel, product, review, score])
        else:
           pass

def get_kakaomakers_review(t1):
    for t in code:
        kakaomakers_review(t)


In [ ]:
get_kakaomakers_review('')

[1841496, '2022-07-12 23:42:09', 'kakaomakers', '매일유업 매일야채 고농축 토마토의 힘/당근의 힘 125ml x 24팩', '맛있어요.', 5]
[509902, '2022-07-12 17:18:31', 'kakaomakers', '매일유업 매일야채 고농축 토마토의 힘/당근의 힘 125ml x 24팩', '한번 주문해서 먹어보니 좋아서 재주문했어요.', 5]
[1174474, '2022-07-12 17:18:26', 'kakaomakers', '매일유업 매일야채 고농축 토마토의 힘/당근의 힘 125ml x 24팩', 'Good  강추합니다', 5]
[1375789, '2022-07-12 16:32:20', 'kakaomakers', '매일유업 매일야채 고농축 토마토의 힘/당근의 힘 125ml x 24팩', '당근은ᆢ저에겐ᆢ너무ᆢ달아요', 2]
[10599852, '2022-07-12 16:20:41', 'kakaomakers', '매일유업 매일야채 고농축 토마토의 힘/당근의 힘 125ml x 24팩', '배송도착과 동시에 마셔봤어요. 진하고 너무 맛있어요. ^^', 5]
[1489936, '2022-07-12 22:25:39', 'kakaomakers', '매일유업 추억의 자판기 우유맛 분말스틱 40포', '맛있는데 좀 많이 달아요', 4]
[1091724, '2022-07-12 13:04:17', 'kakaomakers', '매일유업 추억의 자판기 우유맛 분말스틱 40포', '늦었지만 정말 잘 먹었습니다. 추억의 그 맛 맞네요.', 5]
[9020596, '2022-07-12 16:44:11', 'kakaomakers', '매일유업 매일두유 고단백 190ml*24팩 ', '맛있고 건강하게 먹고 있습니다^^', 5]
[1344967, '2022-07-12 16:22:08', 'kakaomakers', '매일유업 매일두유 고단백 190ml*24팩 ', '믿고먹는 매일두유이번에는 고단백으로구매했습니다 아침에  두유라떼마시면 좋아

#### 🆗제품 리뷰 수집(API)

In [ ]:
# requirements.txt
# pandas>=0.25.3
# requests>=2.22.0
# google-cloud-bigquery>=1.18.0
# pandas_gbq>=0.17.9

In [1]:
# main.py
import pandas as pd
from datetime import datetime, timedelta
import requests
import time 
import re
import json
# from google.cloud import bigquery
import pandas_gbq


# 저장 위치 설정
# client = bigquery.Client()
# project = 'boxwood-chassis-356100'
# dataset_id = 'review'
# table_id = 'review_all_test'
# table_ref = client.dataset(dataset_id).table(table_id)
# Table = client.get_table(table_ref)  # API request


# code 정보 수집
code = []
search_keyword = ['매일유업', '셀렉스', ]
for text_ in search_keyword:
    url_master = 'https://makers-gateway.kakao.com/service/front/v1/search?page=0&query='+text_+'&size=50&on_sale=false'
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.141 Safari/537.36'}
    ka_makers = requests.get(url_master, headers=headers).json()
    for i in range( len(ka_makers['result']['searchItemList']) ):
        category =  ka_makers['result']['searchItemList'][i]['categoryLargeName']
        product = ka_makers['result']['searchItemList'][i]['roundItemName']
        if ((category =='푸드') & (text_ in product) ) :
            code_ = ka_makers['result']['searchItemList'][i]['masterItemId']
            code.append(code_)
        else:
            pass

def kakaomakers_review(code_):
    url = 'https://makers-gateway.kakao.com/service/front/v2/reviews/item/' + str(code_) + '?reviewSort=LATEST&page=1&size=50'
    ka_makers = requests.get(url, headers=headers).json()
    df = pd.json_normalize(ka_makers['result']['contents'])
    df = df[['userId', 'createdAt', 'roundItemName', 'text', 'score', 'roundItemId']]
    df = df.rename(columns ={'userId': "USER", 'roundItemName': "PRODUCT", 'text': "REVIEW", 'score': "SCORE", 'createdAt' : 'DATE', 'roundItemId' : 'GUBUN' })

    df.REVIEW = df.REVIEW.str.replace('\n', ' ')
    df['DATE'] = pd.to_datetime(df['DATE'], format = '%Y-%m-%d' )
    df.GUBUN = df.GUBUN.astype('object')
    df.USER  = df.USER.astype('object')
    return df

# 반복수집 & 일자정렬 
def review_insert(code_list):
    date_start = (datetime.today() + timedelta(hours=9) - timedelta(1)).replace(minute=0, hour=0, second=0).strftime('%Y-%m-%d %H:%M:%S')
    date_end   = (datetime.today() + timedelta(hours=9) - timedelta(0)).replace(minute=0, hour=0, second=0).strftime('%Y-%m-%d %H:%M:%S')
    df_ = pd.DataFrame()
    for code_ in code_list:
        df_ = pd.concat([df_, kakaomakers_review(code_) ], ignore_index=True)
    df_ = df_[ (date_start <= df_['DATE'] ) &  (date_end > df_['DATE'] )].reset_index(drop=True)
    # pandas_gbq.to_gbq(df_, table_name, project_id=project, if_exists = 'append' )
    # return f'Hello World!'
    return df_

In [2]:
df = kakaomakers_review_insert(code)

In [7]:
ka_makers['result']['searchItemList'][1]['categoryLargeName'],
ka_makers['result']['searchItemList'][1]['masterItemId']

100008290

In [8]:
date_start = (datetime.today() + timedelta(hours=9) - timedelta(1)).replace(minute=0, hour=0, second=0).strftime('%Y-%m-%d %H:%M:%S')
date_end   = (datetime.today() + timedelta(hours=9) - timedelta(0)).replace(minute=0, hour=0, second=0).strftime('%Y-%m-%d %H:%M:%S')
date_start, date_end

('2023-05-18 00:00:00', '2023-05-19 00:00:00')

In [ ]:
# main.py
import pandas as pd
from datetime import datetime, timedelta
import requests
import time 
import re
from google.cloud import bigquery


# 저장 위치 설정
client = bigquery.Client()
project = 'boxwood-chassis-356100'
dataset_id = 'review'
table_id = 'review_all'
table_ref = client.dataset(dataset_id).table(table_id)
Table = client.get_table(table_ref)  # API request


# code 정보 수집
code = []
search_keyword = ['매일유업', '셀렉스', ]
for text_ in search_keyword:
    url_master = 'https://makers-gateway.kakao.com/service/front/v1/search?page=0&query='+text_+'&size=50&on_sale=false'
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.141 Safari/537.36'}
    ka_makers = requests.get(url_master, headers=headers).json()
    for i in range( len(ka_makers['result']['searchItemList']) ):
        category =  ka_makers['result']['searchItemList'][i]['categoryLargeName']
        product = ka_makers['result']['searchItemList'][i]['roundItemName']
        if ((category =='푸드') & (text_ in product) ) :
            code_ = ka_makers['result']['searchItemList'][i]['masterItemId']
            code.append(code_)
        else:
            pass

def kakaomakers_review(code_):
    url = 'https://makers-gateway.kakao.com/service/front/v2/reviews/item/' + str(code_) + '?reviewSort=LATEST&page=1&size=50'
    ka_makers = requests.get(url, headers=headers).json()
    ka_makers = ka_makers['result']['contents']
    date_start = (datetime.today() + timedelta(hours=9) - timedelta(1)).replace(minute=0, hour=0, second=0).strftime('%Y-%m-%d %H:%M:%S')
    date_end   = (datetime.today() + timedelta(hours=9) - timedelta(0)).replace(minute=0, hour=0, second=0).strftime('%Y-%m-%d %H:%M:%S')
    for data_ in ka_makers:
        reg_dtm = data_['createdAt']
        if date_start <= reg_dtm  <= date_end :
            user_id = data_['userId']
            channel = 'kakaomakers'
            product = data_['roundItemName']
            review  = data_['text'].replace('\n', ' ')
            score   = int(data_['score']) 
            if score >= 3:
              score = score + 1
            rows_to_insert = [(user_id, channel, product, review, int(score), reg_dtm, code_ )]
            client.insert_rows(Table, rows_to_insert)
        else:
           pass

def get_kakaomakers_review(t1):
    for t in code:
        kakaomakers_review(t)
    return f'Hello World!'